In [53]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from CRUD import CRUD

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "Kofuku"

# Connect to database via CRUD Module
db = CRUD(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#Logo
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), 
             style={'marginLeft': '0px', 'height':'10%', 'width':'10%', 'display': 'inline-block'})],
    href='https://www.snhu.edu'),
    html.Center(html.B(html.H1('CS-340 Mark Williams')),  style={'marginLeft':'400px','display': 'inline-block'}),
    html.Hr(),
    html.Div(
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.Dropdown(['Water Rescue', 'Mountain or Wilderness Rescue', 'Disaster or Individual Tracking', 'Reset'],
                     id='filter_type'),
    ),
    html.Div(id='output-container'),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                        data=df.to_dict('records'),
                        # User-friendly features:
                        sort_action='native',  # Enable client-side sorting
                        filter_action='native', # Enable client-side filtering
                        row_selectable='single', # Allow single row selection
                        selected_rows=[0],  # This line pre-selects the first row (index 0)
                        page_action='native', # Enable client-side pagination
                        page_size=10, # Display 10 rows per page (adjust as needed)
                        style_table={'overflowX': 'auto'}, # Enable horizontal scrolling if needed
                        style_cell={'textAlign': 'center'}, # center-align text in cells

                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    [Output('datatable-id','data'),  # Specify data as an output
     Output('datatable-id','columns')], # Add columns as an output
    [Input('filter_type', 'value')]
)
def update_dashboard(value):
    ## FIX ME Add code to filter interactive data table with MongoDB queries
    if value == 'Water Rescue':
        # MongoDB query to filter for Water Rescue animals
        query = {
            'breed': {'$in': ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
            'sex_upon_outcome': 'Intact Female',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
            }
    elif value == "Mountain or Wilderness Rescue":
        query = {
            'breed': {'$in': ['German Shepherd', 'Alaskan Malamute',
                              'Old English Sheepdog', 'Siberian Husky', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
        }
    elif value == "Disaster or Individual Tracking":
        query = {
            'breed': {'$in': ['German Shepherd', 'Doberman Pinscher',
                             'Golden Retriever', 'Bloodhound', 'Rottweiler']},
            'sex_upon_outcome': 'Intact Male',
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
        }
    else:
        # Load all data if no filter
        query = {} # Use an empty dictionary to load all data

    # Handle cases where db.read() might return None or an empty list
    try:
        data_from_db = db.read(query)
        if data_from_db is None or not data_from_db: # Check if data is None or empty
            df = pd.DataFrame() # Create an empty DataFrame
        else:
            df = pd.DataFrame.from_records(data_from_db)
    except Exception as e:
        print(f"Error fetching data from MongoDB: {e}")
        df = pd.DataFrame() # Create an empty DataFrame in case of error

    # Remove the '_id' column if present
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    # Generate columns based on the DataFrame (even if it's empty)
    columns = [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]

    # Convert the DataFrame to the list of dictionaries format for 'data'
    data = df.to_dict('records')

    return data, columns  # Return data and columns

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    # Ensure viewData is not None and is not empty before creating the DataFrame
    if viewData is None or len(viewData) == 0:
        # Return an empty graph or a placeholder if no data is available
        return dcc.Graph(figure={})

    # Convert the derived_virtual_data (list of dictionaries) to a Pandas DataFrame
    dff = pd.DataFrame(viewData) # Use a new variable name like dff to avoid confusion with the original df

    
    fig = px.pie(dff, names='breed', title='Preferred Animals') # Use dff instead of df

    return [
        dcc.Graph(figure=fig)
    ]

    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
   
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:15027/
